In [ ]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v10.0 - FULL TIP SYNC
Sincroniza TODOS los headers hasta el tip actual
"""

import hashlib
import struct
import socket
import time
import random
import select
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass
from collections import defaultdict
from datetime import datetime

@dataclass
class BlockHeader:
    version: int
    prev_hash: bytes
    merkle_root: bytes
    timestamp: int
    bits: int
    nonce: int

    def serialize(self) -> bytes:
        return (struct.pack('<I', self.version) + self.prev_hash +
                self.merkle_root + struct.pack('<I', self.timestamp) +
                struct.pack('<I', self.bits) + struct.pack('<I', self.nonce))

    def hash(self) -> bytes:
        return hashlib.sha256(hashlib.sha256(self.serialize()).digest()).digest()

    def hash_hex(self) -> str:
        return self.hash()[::-1].hex()

@dataclass
class Block:
    header: BlockHeader
    coinbase_script: bytes = b''

    def serialize(self) -> bytes:
        data = self.header.serialize()
        data += b'\x01'
        data += self._coinbase()
        return data

    def _coinbase(self) -> bytes:
        tx = struct.pack('<I', 1)
        tx += b'\x01' + bytes(32) + struct.pack('<I', 0xffffffff)
        script = self.coinbase_script or b'\x04KAORU'
        tx += bytes([len(script)]) + script + struct.pack('<I', 0xffffffff)
        tx += b'\x01' + struct.pack('<Q', 50 * 10**8)
        tx += b'\x19\x76\xa9\x14' + bytes(20) + b'\x88\xac'
        tx += struct.pack('<I', 0)
        return tx


class KaoruBridgeV10:
    """
    v10.0 - FULL TIP SYNC
    Sincroniza TODOS los ~875,000 headers para construir sobre el tip REAL
    """

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_BLOCK = 2

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.our_blocks: List[Block] = []
        self.stats = defaultdict(int)
        self.headers_count = 0
        self.tip_header: Optional[bytes] = None
        self.tip_height: int = 0

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    def log(self, msg: str):
        t = datetime.now().strftime("%H:%M:%S")
        print(f"      [{t}] {msg}")

    def msg(self, cmd: str, payload: bytes) -> bytes:
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip: str) -> bytes:
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x11/KaoruBridge:10.0/'
        p += struct.pack('<i', 900000)
        p += b'\x01'
        return p

    def getheaders_payload(self, locator_hash: bytes) -> bytes:
        p = struct.pack('<I', 70016)
        p += b'\x01'
        p += locator_hash
        p += bytes(32)
        return p

    def read_varint(self, data: bytes, offset: int) -> Tuple[int, int]:
        first = data[offset]
        if first < 0xfd:
            return first, offset + 1
        elif first == 0xfd:
            return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
        else:
            return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5

    def parse_msgs(self, data: bytes) -> List[Tuple[str, bytes]]:
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def recv_all(self, sock: socket.socket, timeout: float = 10.0) -> bytes:
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(262144)
                    if chunk:
                        data += chunk
                    else:
                        break
            except:
                break
        sock.setblocking(True)
        return data

    def sync_all_headers(self, ip: str) -> bool:
        """Sincroniza TODOS los headers hasta el tip."""

        self.log(f"🔄 SINCRONIZACIÓN COMPLETA desde {ip}")
        self.log(f"   Esto tomará ~2-5 minutos para ~875,000 headers...")

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(60)
            sock.connect((ip, 8333))

            # Handshake
            sock.send(self.msg('version', self.version_payload(ip)))
            time.sleep(0.5)

            data = self.recv_all(sock, 3.0)
            peer_height = 0

            for cmd, payload in self.parse_msgs(data):
                if cmd == 'version':
                    if len(payload) >= 85:
                        peer_height = struct.unpack('<i', payload[81:85])[0]
                        # Fix: a veces el height viene mal, usar valor razonable
                        if peer_height < 0 or peer_height > 2000000:
                            peer_height = 875000
                        self.log(f"   Peer altura: {peer_height:,}")
                    sock.send(self.msg('verack', b''))
                elif cmd == 'ping':
                    sock.send(self.msg('pong', payload[:8]))

            time.sleep(0.3)

            # Genesis como punto de partida
            current_tip = bytes.fromhex("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f")[::-1]

            total_headers = 0
            last_header = None
            start_time = time.time()

            while True:
                sock.send(self.msg('getheaders', self.getheaders_payload(current_tip)))

                data = self.recv_all(sock, 15.0)

                headers_this_round = 0

                for cmd, payload in self.parse_msgs(data):
                    if cmd == 'headers':
                        if not payload:
                            continue
                        count, offset = self.read_varint(payload, 0)

                        if count == 0:
                            self.log(f"   ✅ Sincronización completa!")
                            self.tip_header = last_header
                            self.tip_height = total_headers
                            sock.close()
                            return True

                        for _ in range(count):
                            if offset + 81 > len(payload):
                                break
                            header_data = payload[offset:offset+80]
                            current_tip = self.double_sha256(header_data)
                            last_header = header_data
                            offset += 81
                            headers_this_round += 1

                        total_headers += headers_this_round
                        self.headers_count = total_headers

                        # Progreso cada 50,000 headers
                        if total_headers % 50000 < 2000:
                            elapsed = time.time() - start_time
                            rate = total_headers / elapsed if elapsed > 0 else 0
                            eta = (peer_height - total_headers) / rate if rate > 0 else 0
                            self.log(f"   📊 {total_headers:,} headers ({total_headers*100//peer_height}%) - ETA: {eta:.0f}s")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                if headers_this_round == 0:
                    # No más headers
                    break

                if headers_this_round < 2000:
                    # Llegamos al tip
                    self.log(f"   ✅ Llegamos al tip!")
                    break

            self.tip_header = last_header
            self.tip_height = total_headers

            tip_hash = self.double_sha256(last_header)[::-1].hex() if last_header else "N/A"
            elapsed = time.time() - start_time

            self.log(f"   📋 Sincronización completa:")
            self.log(f"      Headers: {total_headers:,}")
            self.log(f"      Tip: {tip_hash[:32]}...")
            self.log(f"      Tiempo: {elapsed:.1f}s")

            sock.close()
            return True

        except Exception as e:
            self.log(f"   ❌ Error: {e}")
            return False

    def create_block(self) -> Block:
        """Crea bloque sobre el tip real."""

        prev_hash = self.double_sha256(self.tip_header)
        prev_bits = struct.unpack('<I', self.tip_header[72:76])[0]

        script = f"KAORU_v10_HEIGHT_{self.tip_height}".encode()
        merkle = self.double_sha256(script)

        header = BlockHeader(
            version=0x20000000,
            prev_hash=prev_hash,
            merkle_root=merkle,
            timestamp=int(time.time()),
            bits=prev_bits,
            nonce=0
        )

        return Block(header=header, coinbase_script=script)

    def mine_block(self, block: Block, target_zeros: int = 4) -> bool:
        target = "0" * target_zeros
        start = time.time()

        for nonce in range(100_000_000):
            block.header.nonce = nonce
            h = block.header.hash_hex()

            if h.startswith(target):
                elapsed = time.time() - start
                self.log(f"   ✅ Minado! Hash: {h[:32]}...")
                self.log(f"      Nonce: {nonce:,} | Tiempo: {elapsed:.2f}s")
                return True

            if nonce % 5_000_000 == 0 and nonce > 0:
                self.log(f"      ... {nonce:,} hashes")

        return False

    def propagate(self, ip: str, block: Block, duration: int = 90) -> Dict:
        result = {
            'ip': ip,
            'headers_sent': 0,
            'blocks_sent': 0,
            'getdata_received': False,
            'reject_received': False,
            'reject_reason': ''
        }

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, 8333))
            start = time.time()

            self.log("→ Conectado!")
            sock.send(self.msg('version', self.version_payload(ip)))

            inv_sent = False

            while time.time() - start < duration:
                data = self.recv_all(sock, 3.0)

                if not data:
                    time.sleep(0.3)
                    continue

                for cmd, payload in self.parse_msgs(data):
                    self.stats[f'recv_{cmd}'] += 1

                    if cmd == 'version':
                        try:
                            ua_len = payload[80]
                            agent = payload[81:81+ua_len].decode('ascii', errors='ignore')
                            self.log(f"← VERSION ({agent[:40]})")
                        except:
                            pass
                        sock.send(self.msg('verack', b''))

                    elif cmd == 'verack':
                        self.log("← VERACK ✓")

                        if not inv_sent:
                            time.sleep(0.3)
                            inv = b'\x01' + struct.pack('<I', self.MSG_BLOCK) + block.header.hash()
                            sock.send(self.msg('inv', inv))
                            inv_sent = True
                            self.log(f"→ INV (bloque altura ~{self.tip_height + 1})")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))
                        self.log("← PING → PONG")

                    elif cmd == 'sendcmpct':
                        sock.send(self.msg('sendcmpct', b'\x00' + struct.pack('<Q', 2)))

                    elif cmd == 'getheaders':
                        self.log("← GETHEADERS ⭐")
                        headers = b'\x01' + block.header.serialize() + b'\x00'
                        sock.send(self.msg('headers', headers))
                        result['headers_sent'] += 1
                        self.log(f"→ HEADERS (1 - altura ~{self.tip_height + 1})")

                    elif cmd == 'getdata':
                        self.log("← GETDATA ⭐⭐⭐ ¡¡¡PIDEN NUESTRO BLOQUE!!!")
                        result['getdata_received'] = True

                        count, offset = self.read_varint(payload, 0)

                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            if inv_hash == block.header.hash():
                                sock.send(self.msg('block', block.serialize()))
                                result['blocks_sent'] += 1
                                self.log("→ BLOCK 📦📦📦 ¡¡ENVIADO!!")

                    elif cmd == 'reject':
                        result['reject_received'] = True
                        try:
                            msg_len = payload[0]
                            msg_type = payload[1:1+msg_len].decode()
                            code = payload[1+msg_len]
                            reason_len = payload[2+msg_len]
                            reason = payload[3+msg_len:3+msg_len+reason_len].decode()
                            result['reject_reason'] = reason
                            self.log(f"← REJECT ❌ {msg_type}: {reason} (code {code})")
                        except:
                            self.log("← REJECT ❌")

                    elif cmd == 'wtxidrelay':
                        sock.send(self.msg('wtxidrelay', b''))

                    elif cmd == 'inv':
                        count, _ = self.read_varint(payload, 0) if payload else (0, 0)
                        self.log(f"← INV ({count} items)")

            sock.close()

        except Exception as e:
            result['error'] = str(e)

        return result

    def discover_nodes(self) -> List[str]:
        nodes = []
        for seed in ['seed.bitcoin.sipa.be', 'dnsseed.bluematt.me', 'seed.bitcoinstats.com']:
            try:
                nodes.extend(socket.gethostbyname_ex(seed)[2][:10])
            except:
                pass
        random.shuffle(nodes)
        return nodes

    def execute(self, num_peers: int = 3):
        print(f"""
╔══════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                          ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗  ██████╗     ██████╗      ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║ ██╔═████╗   ██╔═████╗     ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║ ██║██╔██║   ██║██╔██║     ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║ ████╔╝██║   ████╔╝██║     ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║ ╚██████╔╝██╗╚██████╔╝     ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝  ╚═════╝ ╚═╝ ╚═════╝      ║
║                                                                                          ║
║                              FULL TIP SYNCHRONIZATION                                    ║
║                     "Sincroniza TODOS los ~875,000 headers"                              ║
║                                                                                          ║
╚══════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        print("=" * 100)
        print("[FASE 1] 🌐 DESCUBRIENDO NODOS")
        print("=" * 100)

        nodes = self.discover_nodes()
        print(f"\n   Encontrados: {len(nodes)} nodos")

        print("\n" + "=" * 100)
        print("[FASE 2] 🔗 SINCRONIZANDO TODOS LOS HEADERS (~875,000)")
        print("=" * 100 + "\n")

        for ip in nodes[:3]:
            if self.sync_all_headers(ip):
                break

        if not self.tip_header:
            print("\n   ❌ No se pudo sincronizar")
            return

        tip_hash = self.double_sha256(self.tip_header)[::-1].hex()
        print(f"\n   📋 TIP REAL OBTENIDO:")
        print(f"      Altura: {self.tip_height:,}")
        print(f"      Hash: {tip_hash}")

        print("\n" + "=" * 100)
        print("[FASE 3] ⛏️  CREANDO BLOQUE SOBRE EL TIP REAL")
        print("=" * 100 + "\n")

        block = self.create_block()

        print(f"   Bloque en altura: {self.tip_height + 1:,}")
        print(f"   prev_hash: {block.header.prev_hash[::-1].hex()[:48]}...")
        print(f"\n   ⛏️ Minando...")

        if self.mine_block(block, target_zeros=4):
            self.our_blocks.append(block)
        else:
            return

        print("\n" + "=" * 100)
        print("[FASE 4] 📡 PROPAGANDO BLOQUE AL TIP")
        print("=" * 100)

        results = []

        for ip in nodes[:num_peers]:
            print(f"\n   ╔{'═'*90}╗")
            print(f"   ║  📡 {ip:82} ║")
            print(f"   ╠{'═'*90}╣")

            result = self.propagate(ip, block, 90)
            results.append(result)

            status = "✅ BLOQUE ENVIADO!" if result['blocks_sent'] > 0 else \
                     "⚠️ REJECT" if result['reject_received'] else "❌"

            print(f"   ╠{'═'*90}╣")
            print(f"   ║  Headers: {result['headers_sent']} | Blocks: {result['blocks_sent']} | GETDATA: {'✅' if result['getdata_received'] else '❌'} | Status: {status:30} ║")
            if result['reject_reason']:
                print(f"   ║  Reject reason: {result['reject_reason']:72} ║")
            print(f"   ╚{'═'*90}╝")

        print("\n" + "=" * 100)
        print("                              📊 RESULTADOS FINALES")
        print("=" * 100)

        total_blocks = sum(r['blocks_sent'] for r in results)
        getdata_count = sum(1 for r in results if r['getdata_received'])
        reject_count = sum(1 for r in results if r['reject_received'])

        print(f"""
   ╔══════════════════════════════════════════════════════════════════════════════════════════╗
   ║                                                                                          ║
   ║   Headers sincronizados:    {self.tip_height:>10,}                                               ║
   ║   Bloque creado en altura:  {self.tip_height + 1:>10,}                                               ║
   ║   GETDATA recibidos:        {getdata_count:>10}                                               ║
   ║   BLOQUES ENVIADOS:         {total_blocks:>10}                                               ║
   ║   REJECT recibidos:         {reject_count:>10}                                               ║
   ║                                                                                          ║
   ╚══════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        if total_blocks > 0:
            print("   🎉🎉🎉 ¡¡¡ÉXITO!!! ¡¡¡BLOQUE ENVIADO A MAINNET!!! 🎉🎉🎉")
        elif getdata_count > 0:
            print("   ⚠️ Los nodos pidieron el bloque pero hubo un problema al enviarlo")
        elif reject_count > 0:
            print("   ❌ Los nodos rechazaron el bloque (PoW insuficiente)")
        else:
            print("   ❌ Los nodos no solicitaron el bloque")

        return results


if __name__ == "__main__":
    bridge = KaoruBridgeV10()
    bridge.execute(num_peers=3)